# SQLAlchemy
Is a library that facilitates the communication between Python programs and databases. SQLAlchemy provides a standard interface that allows developers to create database-agnostic code to communicate with a wide variety of database engines.

In [1]:
import sqlalchemy as db

from sqlalchemy import create_engine,inspect

import pandas as pd

In [2]:

engine = db.create_engine('sqlite:///test.sqlite') #Create test.sqlite automatically
connection = engine.connect()

# Holds a collection of Table objects
# The Table objects in the collection and their columns may participate in implicit SQL execution.
metadata = db.MetaData()

emp = db.Table('emp', metadata,
              db.Column('Id', db.Integer()),
              db.Column('name', db.String(255), nullable=False),
              db.Column('salary', db.Float(), default=100.0),
              db.Column('active', db.Boolean(), default=True)
              )

metadata.create_all(engine) #Creates the table

In [3]:
# metadata provides the basic and relevant information about the database objects.
employeeT = db.Table('emp',metadata)

# Print the column names
print(employeeT.columns.keys())

['Id', 'name', 'salary', 'active']


In [4]:
#this is used to look at the schema of elements in a database
# Get Database Information
inspector = inspect(engine)

# key is the name of the column and value is the attribute for that field
print(inspector.get_columns('emp'))

[{'name': 'Id', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'name', 'type': VARCHAR(length=255), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'salary', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'active', 'type': BOOLEAN(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]


## Inserting Data using SQLAlchemy

In [5]:
# Inserting many records at ones

query = db.insert(emp) 
values_list = [{'Id':'2', 'name':'ram', 'salary':80000, 'active':False},
               {'Id':'3', 'name':'ramesh', 'salary':70000, 'active':True}]
ResultProxy = connection.execute(query,values_list)

In [6]:
# insert a row
engine.execute('INSERT INTO "emp" '
               '(id, name, salary, active) '
               'VALUES (4,"Sandy",40000,True)')


In [7]:
result = connection.execute('SELECT * FROM emp')
dfNew = pd.DataFrame(result)
dfNew.columns = results[0].keys()
dfNew.head()

NameError: name 'results' is not defined

In [ ]:
'''

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df

'''

## Selecting rows using SQLAlchemy sintax

In [ ]:

#Equivalent to 'SELECT * FROM table_name'
query = db.select([emp])
ResultProxy = connection.execute(query)

# The ResultProxy object return information of rows
ResultSet = ResultProxy.fetchall()

df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df.head()


In [ ]:
# Get the number of Rows and columns
df.shape

### Close de connection and release memory

In [ ]:
ResultProxy.close()
connection.close()

## Selecting records using Pandas

We can use the pandas read_sql_query function to read the results of a SQL query directly into a pandas DataFrame. The below code will execute the same query that we just did, but it will return a DataFrame. It has several advantages over the query we did above:

- It doesn’t require us to create a Cursor object or call fetchall at the end.
- It automatically reads in the names of the headers from the table.
- It creates a DataFrame, so we can quickly explore the data.

In [ ]:
#get all the rows from the "test" table in the database
sql = "SELECT id, name, salary, active FROM emp;"

#the read_sql function takes in the SQL statement for the information requested and the engine (to connect to database)
data_df = pd.read_sql(sql, engine)
data_df

## Open a Database using Pandas

In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("flights.db")


In [ ]:
### Names of tables in the database
dfTables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
dfTables

## Selecting rows from an existing database

In [ ]:
df = pd.read_sql_query("select *  from airlines;", conn)
df.head()

In [ ]:
# Number of rows and columns 
df.shape


In [ ]:
# Selecting only the columns we want to display
df = pd.read_sql_query("select id, name, alias, country, active  from airlines;", conn)
df.head()

In [ ]:
 pd.read_sql_query("select id, name, alias, country, active  from airlines limit 3;", conn)

## Inserting rows with Pandas

In [ ]:
# We specify 9 values to insert, one for each column in airlines. This will add a new row to the table.
# Cursors allow us to execute SQL queries against a database:
cur = conn.cursor()
cur.execute("insert into airlines values (6048, 19846, 'Test flight', '', '', null, null, null, 'Y')")

- By default, sqlite3 opens a transaction when you do any query that modifies the database.
- SQLite doesn’t write to the database until you commit a transaction. A transaction consists of 1 or more queries that all make changes to the database at once. This is designed to make it easier to recover from accidental changes, or errors. Transactions allow you to run several queries, then finally alter the database with the results of all of them

In [ ]:
conn.commit()

In [ ]:
# Checking if the record was inserted
pd.read_sql_query("select id, name, alias, country, active  from airlines where id=19846;", conn)

## Inserting Rows passing parameters into query
- Most of the time, when you insert data into a database, it won’t be hardcoded, it will be dynamic values you want to pass in. These dynamic values might come from downloaded data, or might come from user input.

- When working with dynamic data, it might be tempting to insert values using Python string formatting

In [ ]:
# Checking if the record exists
pd.read_sql_query("select id, name, country, active from airlines where id=19847;", conn)

**Any ?** value in the query will be replaced by a value in values. The first **?** will be replaced by the first item in values, the second by the second, and so on. This works for any type of query.

In [ ]:
cur = conn.cursor()
values = (6049, 19847,'Test Flight', 'Y')
cur.execute("insert into airlines values (?, ?, ?, '', '', null, null, null, ?)", values)
conn.commit()

In [ ]:
# Checking if the record was inserted
pd.read_sql_query("select id, name, country, active from airlines where id=19847;", conn)

## Updating rows
We can modify rows in a SQLite table using the execute method.

In [ ]:
pd.read_sql_query("select  id, name, country, active from airlines where id=19847;", conn)

In [ ]:
cur = conn.cursor()
values = ('New Airlines','USA', 19847)
cur.execute("update airlines set name=?, country=? where id=?", values)
conn.commit()

In [ ]:
pd.read_sql_query("select id, name, country, active from airlines where id=19847;", conn)

## Deleting rows
Finally, we can delete the rows in a database using the execute method

In [ ]:
pd.read_sql_query("select id, name, country, active from airlines where id=19847;", conn)

In [ ]:
cur = conn.cursor()
values = (19847,)
cur.execute("delete from airlines where id=?", values)
conn.commit()

In [ ]:
pd.read_sql_query("select * from airlines where id=19847;", conn)

## Altering tables with Pandas

In [ ]:
#  Adding a new column: airplanes field to the airlines table that indicates how many airplanes each airline owns
#  Cursors allow us to execute SQL queries against a database:
cur = conn.cursor()
cur.execute("alter table airlines add column airplanes integer;")

In [ ]:
pd.read_sql_query("select * from airlines limit 1;", conn)

## Creating tables
We can create tables by executing a SQL query. We can create a table to represent each daily flight on a route, with the following columns:

- id — integer
- departure — date, when the flight left the airport
- arrival — date, when the flight arrived at the destination
- number — text, the flight number
- route_id — integer, the id of the route the flight was flying

In [ ]:
cur = conn.cursor()
cur.execute("create table daily_flights (id integer, departure date, arrival date, number text, route_id integer)")
conn.commit()


In [ ]:
# Once we create a table, we can insert data into it normally:

cur.execute("insert into daily_flights values (1, '2016-09-28 0:00', '2016-09-28 12:00', 'T1', 1)")
conn.commit()

In [ ]:
# Select data from new table
pd.read_sql_query("select id, departure, arrival, number, route_id from daily_flights;", conn)

In [ ]:

conn.close()